In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
TEST_DURATION = 5

In [ ]:
from IPython.display import Audio
from pymu.core.old import generate_waveform

# Generate a sine wave at 440Hz (A4 note)
sample_rate = 44100
samples = generate_waveform(frequency=440, duration=TEST_DURATION, sample_rate=sample_rate)

# Play through speakers
Audio(samples, rate=sample_rate)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

# Compute the FFT of the samples
fft_vals = np.fft.rfft(samples)
fft_freqs = np.fft.rfftfreq(len(samples), 1/sample_rate)

# Find the frequency with the highest magnitude
peak_freq = fft_freqs[np.argmax(np.abs(fft_vals))]

print(f"Peak frequency: {peak_freq:.2f} Hz")

# Plot the frequency spectrum
plt.figure(figsize=(10, 4))
plt.plot(fft_freqs, np.abs(fft_vals))
plt.title("Frequency Spectrum of the Waveform")
plt.xlabel("Frequency (Hz)")
plt.ylabel("Magnitude")
plt.xlim(0, 1000)
plt.grid(True)
plt.show()

In [ ]:
from pymu.core.old import frequency_modulation, create_harmonic_series
from IPython.display import Audio

# Example 1: Frequency Modulation Synthesis
fm_samples = frequency_modulation(
    carrier_freq=440,        # Carrier frequency (A4)
    modulator_freq=220,      # Modulator frequency
    modulation_index=2.0,    # Modulation index
    duration=TEST_DURATION,            # Duration in seconds
    sample_rate=sample_rate  # Use the existing sample_rate variable
)
print("Frequency Modulation Example (A4 carrier, 220Hz modulator):")
display(Audio(fm_samples, rate=sample_rate))

# Example 2: Harmonic Series Synthesis
harmonic_samples = create_harmonic_series(
    fundamental_freq=440,           # Fundamental frequency (A4)
    num_harmonics=5,         # Number of harmonics
    amplitudes=[1, 0.5, 0.3, 0.2, 0.1],  # Amplitudes for each harmonic
    duration=TEST_DURATION,            # Duration in seconds
    sample_rate=sample_rate  # Use the existing sample_rate variable
)
print("Harmonic Series Example (A4, 5 harmonics):")
display(Audio(harmonic_samples, rate=sample_rate))

In [ ]:
from scipy.signal import sawtooth
from IPython.display import Audio

# Generate a bassy sawtooth wave (e.g., 55 Hz, A1)
bass_freq = 55  # Hz
t = np.linspace(0, TEST_DURATION, int(sample_rate * TEST_DURATION), endpoint=False)
bassy_saw = sawtooth(2 * np.pi * bass_freq * t)

print("Bassy Sawtooth Wave (A1, 55Hz):")
display(Audio(bassy_saw, rate=sample_rate))

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
from pymu.core.old import generate_waveform, create_harmonic_series
import numpy as np
import sounddevice as sd
import threading
import time

# First, make sure you have sounddevice installed
# !pip install sounddevice

# Define musical notes and corresponding keyboard keys
keys_to_notes = {
    'a': 'C4', 'w': 'C#4', 's': 'D4', 'e': 'D#4', 'd': 'E4', 'f': 'F4',
    't': 'F#4', 'g': 'G4', 'y': 'G#4', 'h': 'A4', 'u': 'A#4', 'j': 'B4', 'k': 'C5'
}

# Define frequencies for each note
note_frequencies = {
    'C4': 261.63, 'C#4': 277.18, 'D4': 293.66, 'D#4': 311.13, 'E4': 329.63,
    'F4': 349.23, 'F#4': 369.99, 'G4': 392.00, 'G#4': 415.30, 'A4': 440.00,
    'A#4': 466.16, 'B4': 493.88, 'C5': 523.25
}

# Track which notes are currently playing
active_notes = {}
stop_flags = {}
sample_rate = 44100

def play_note(note, waveform_type="harmonic", stop_flag=None):
    freq = note_frequencies[note]
    duration = 1.0  # Shorter duration for testing
    
    if waveform_type == "harmonic":
        samples = create_harmonic_series(
            fundamental_freq=freq, 
            num_harmonics=5,
            amplitudes=[1.0, 0.5, 0.25, 0.125, 0.0625],
            duration=duration,
            sample_rate=sample_rate
        )
    else:
        samples = generate_waveform(freq, duration, waveform=waveform_type, sample_rate=sample_rate)
    
    # Use simpler playback method for testing
    stream = sd.play(samples, sample_rate)
    return stream

# # Function to play a note in a separate thread
# def play_note(note, waveform_type="harmonic", stop_flag=None):
#     freq = note_frequencies[note]
#     duration = 2.0  # Long duration, will be interrupted when key is released
    
#     if waveform_type == "sine":
#         samples = generate_waveform(freq, duration, sample_rate=sample_rate)
#     elif waveform_type == "harmonic":
#         samples = create_harmonic_series(
#             fundamental_freq=freq, 
#             num_harmonics=5,
#             amplitudes=[1.0, 0.5, 0.25, 0.125, 0.0625],
#             duration=duration,
#             sample_rate=sample_rate
#         )
#     else:
#         # You could add more waveform types here
#         samples = generate_waveform(freq, duration, waveform=waveform_type, sample_rate=sample_rate)
    
#     # Start playing the sound
#     stream = sd.OutputStream(samplerate=sample_rate, channels=1, callback=lambda *args: callback(samples, *args, stop_flag=stop_flag))
#     stream.start()
#     return stream

# Callback for sounddevice to get audio data
def callback(samples, outdata, frames, time, status, stop_flag=None):
    if stop_flag and stop_flag.is_set():
        # Fill with zeros to stop the sound
        outdata[:] = np.zeros((frames, 1))
        raise sd.CallbackStop
    else:
        # Use position as an attribute of the callback function instead of global
        if not hasattr(callback, "position"):
            callback.position = 0
        
        # Check for end of samples and wrap around
        if callback.position + frames > len(samples):
            # Calculate how many samples we need from the start
            remaining = callback.position + frames - len(samples)
            # Fill with wrapped-around samples
            outdata[:frames-remaining, 0] = samples[callback.position:]
            outdata[frames-remaining:, 0] = samples[:remaining]
            callback.position = remaining
        else:
            # Normal case - just copy the samples
            outdata[:, 0] = samples[callback.position:callback.position+frames]
            callback.position += frames

# Create an output area for showing pressed keys
output = widgets.Output()

# Define the waveform selector
waveform_selector = widgets.Dropdown(
    options=['sine', 'square', 'triangle', 'sawtooth', 'harmonic'],
    value='harmonic',
    description='Waveform:',
)

# Create a keyboard visualization
keyboard_html = widgets.HTML(
    value="""
    <div style="font-family: monospace; white-space: pre; line-height: 1; font-size: 24px; font-weight: bold;">
      ┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
      │ A │ W │ S │ E │ D │ F │ T │ G │ Y │ H │
      │ C │C#/│ D │D#/│ E │ F │F#/│ G │G#/│ A │
      │   │Db │   │Eb │   │   │Gb │   │Ab │   │
      └───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘
    </div>
    """
)

class KeyboardWidget(widgets.DOMWidget):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._keyboard_handler = self._create_keyboard_handler()
        display(self._keyboard_handler)
    
    def _create_keyboard_handler(self):
        text = widgets.Text(
            placeholder='Click here, then type keys A-W-S-E-D-F-T-G-Y-H-U-J-K to play notes',
            layout=widgets.Layout(width='500px')
        )
        
        def handle_keydown(widget):
            # Check if value exists and is a string before using lower()
            if not hasattr(widget, 'value') or widget.value is None or widget.value == '':
                return
            key = widget.value.lower()
            widget.value = ''  # Clear the input
            
            if key in keys_to_notes and keys_to_notes[key] not in active_notes:
                note = keys_to_notes[key]
                with output:
                    clear_output(wait=True)
                    print(f"Playing: {note} ({note_frequencies[note]:.2f} Hz)")
                
                # Set up a stop flag for this note
                stop_flag = threading.Event()
                stop_flags[note] = stop_flag
                
                # Start playing the note
                stream = play_note(note, waveform_type=waveform_selector.value, stop_flag=stop_flag)
                active_notes[note] = stream
        
        def handle_keyup(widget):
            # This is more complex - we need to determine which key was released
            # In a real app, we'd use JavaScript for this, but for simplicity, 
            # we'll just stop the most recently played note
            if active_notes:
                note = list(active_notes.keys())[-1]
                if note in stop_flags:
                    stop_flags[note].set()
                if note in active_notes:
                    active_notes[note].stop()
                    del active_notes[note]
                    del stop_flags[note]
        
        text.observe(handle_keydown, 'value')
        
        return text

# Display the widgets
display(keyboard_html)
display(waveform_selector)
keyboard = KeyboardWidget()
display(output)

HTML(value='\n    <div style="font-family: monospace; white-space: pre; line-height: 1; font-size: 24px; font-…

Dropdown(description='Waveform:', index=4, options=('sine', 'square', 'triangle', 'sawtooth', 'harmonic'), val…

Text(value='', layout=Layout(width='500px'), placeholder='Click here, then type keys A-W-S-E-D-F-T-G-Y-H-U-J-K…

Output()

In [30]:
# Add this to a cell and run it to test basic sounddevice output
def test_sound():
    print("Testing sound output...")
    samples = generate_waveform(440, 1.0, sample_rate=sample_rate)
    sd.play(samples, sample_rate)
    sd.wait()
    print("Sound test complete.")

test_sound()

Testing sound output...
Sound test complete.


In [31]:
# List audio devices to check configuration
print(sd.query_devices())

   0 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
<  1 Realtek Digital Output (Realtek, MME (0 in, 2 out)
   2 Dell AW3821DW (NVIDIA High Defi, MME (0 in, 2 out)
   3 Primary Sound Driver, Windows DirectSound (0 in, 8 out)
   4 Realtek Digital Output (Realtek USB Audio), Windows DirectSound (0 in, 2 out)
   5 Dell AW3821DW (NVIDIA High Definition Audio), Windows DirectSound (0 in, 2 out)
   6 Dell AW3821DW (NVIDIA High Definition Audio), Windows WASAPI (0 in, 2 out)
   7 Realtek Digital Output (Realtek USB Audio), Windows WASAPI (0 in, 8 out)
   8 Output (NVIDIA High Definition Audio), Windows WDM-KS (0 in, 2 out)
   9 Headphones (), Windows WDM-KS (0 in, 2 out)
  10 Headphones (), Windows WDM-KS (0 in, 2 out)
  11 Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(W-KING D8)), Windows WDM-KS (0 in, 1 out)
  12 Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hands-Free%0
;(W-KING D8)), Windows WDM-KS (1 in, 0 out)
  13 Headset (@System32\drivers\bthhfenum.sys,#2;%1 Hand

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
from pymu.core.old import generate_waveform, create_harmonic_series
import numpy as np
import sounddevice as sd
import time

# Define musical notes and corresponding keyboard keys
keys_to_notes = {
    'a': 'C4', 'w': 'C#4', 's': 'D4', 'e': 'D#4', 'd': 'E4', 'f': 'F4',
    't': 'F#4', 'g': 'G4', 'y': 'G#4', 'h': 'A4', 'u': 'A#4', 'j': 'B4', 'k': 'C5'
}

# Define frequencies for each note
note_frequencies = {
    'C4': 261.63, 'C#4': 277.18, 'D4': 293.66, 'D#4': 311.13, 'E4': 329.63,
    'F4': 349.23, 'F#4': 369.99, 'G4': 392.00, 'G#4': 415.30, 'A4': 440.00,
    'A#4': 466.16, 'B4': 493.88, 'C5': 523.25
}

sample_rate = 44100
output = widgets.Output()

# Define the waveform selector
waveform_selector = widgets.Dropdown(
    options=['sine', 'square', 'triangle', 'sawtooth', 'harmonic'],
    value='harmonic',
    description='Waveform:',
)

# Create a keyboard visualization
keyboard_html = widgets.HTML(
    value="""
    <div style="font-family: monospace; white-space: pre; line-height: 1; font-size: 24px; font-weight: bold;">
      ┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
      │ A │ W │ S │ E │ D │ F │ T │ G │ Y │ H │
      │ C │C#/│ D │D#/│ E │ F │F#/│ G │G#/│ A │
      │   │Db │   │Eb │   │   │Gb │   │Ab │   │
      └───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘
    </div>
    """
)

# Create buttons instead of relying on keyboard events
buttons = {}
button_layout = widgets.Layout(width='40px', height='40px')
button_box = widgets.HBox()

for key, note in keys_to_notes.items():
    button = widgets.Button(
        description=key.upper(),
        layout=button_layout,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
    )
    
    def create_handler(note_name):
        def handle_click(b):
            with output:
                clear_output(wait=True)
                print(f"Playing: {note_name} ({note_frequencies[note_name]:.2f} Hz)")
            
            # Generate and play the sound
            if waveform_selector.value == "harmonic":
                samples = create_harmonic_series(
                    fundamental_freq=note_frequencies[note_name],
                    num_harmonics=5,
                    amplitudes=[1.0, 0.5, 0.25, 0.125, 0.0625],
                    duration=1.0,
                    sample_rate=sample_rate
                )
            else:
                samples = generate_waveform(
                    frequency=note_frequencies[note_name],
                    duration=1.0,
                    waveform=waveform_selector.value,
                    sample_rate=sample_rate
                )
            
            # Play sound with explicit wait=False to make it non-blocking
            sd.play(samples, sample_rate, blocking=False)
        
        return handle_click
    
    button.on_click(create_handler(note))
    buttons[key] = button
    button_box.children = (*button_box.children, button)

# Display the interface
display(keyboard_html)
display(waveform_selector)
display(button_box)
display(output)

# Also provide a text input method with additional debug output
text_input = widgets.Text(
    placeholder='Or type keys here: A-W-S-E-D-F-T-G-Y-H-U-J-K',
    layout=widgets.Layout(width='500px')
)

def on_text_change(change):
    if 'new' in change and change['new']:
        key = change['new'].lower()
        text_input.value = ''  # Clear the input
        
        with output:
            print(f"Key pressed: {key}")  # Debug output
        
        if key in keys_to_notes:
            note = keys_to_notes[key]
            with output:
                print(f"Playing: {note} ({note_frequencies[note]:.2f} Hz)")
            
            # Generate and play the sound
            if waveform_selector.value == "harmonic":
                samples = create_harmonic_series(
                    fundamental_freq=note_frequencies[note],
                    num_harmonics=5,
                    amplitudes=[1.0, 0.5, 0.25, 0.125, 0.0625],
                    duration=1.0,
                    sample_rate=sample_rate
                )
            else:
                samples = generate_waveform(
                    frequency=note_frequencies[note],
                    duration=1.0,
                    waveform=waveform_selector.value,
                    sample_rate=sample_rate
                )
            
            # Debug output
            print(f"Generated {len(samples)} samples, max amplitude: {np.max(np.abs(samples))}")
            
            # Play sound with explicit wait=False to make it non-blocking
            sd.play(samples, sample_rate, blocking=False)

text_input.observe(on_text_change, names='value')
display(text_input)

HTML(value='\n    <div style="font-family: monospace; white-space: pre; line-height: 1; font-size: 24px; font-…

Dropdown(description='Waveform:', index=4, options=('sine', 'square', 'triangle', 'sawtooth', 'harmonic'), val…

Output()

Text(value='', layout=Layout(width='500px'), placeholder='Or type keys here: A-W-S-E-D-F-T-G-Y-H-U-J-K')

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
from pymu.core.old import generate_waveform, create_harmonic_series
import numpy as np
import sounddevice as sd
import time

# Define musical notes and corresponding keyboard keys
keys_to_notes = {
    'a': 'C4', 'w': 'C#4', 's': 'D4', 'e': 'D#4', 'd': 'E4', 'f': 'F4',
    't': 'F#4', 'g': 'G4', 'y': 'G#4', 'h': 'A4', 'u': 'A#4', 'j': 'B4', 'k': 'C5'
}

# Define frequencies for each note
note_frequencies = {
    'C4': 261.63, 'C#4': 277.18, 'D4': 293.66, 'D#4': 311.13, 'E4': 329.63,
    'F4': 349.23, 'F#4': 369.99, 'G4': 392.00, 'G#4': 415.30, 'A4': 440.00,
    'A#4': 466.16, 'B4': 493.88, 'C5': 523.25
}

sample_rate = 44100
output = widgets.Output()

# Define the waveform selector
waveform_selector = widgets.Dropdown(
    options=['sine', 'square', 'triangle', 'sawtooth', 'harmonic'],
    value='harmonic',
    description='Waveform:',
)

# Create a keyboard visualization
keyboard_html = widgets.HTML(
    value="""
    <div style="font-family: monospace; white-space: pre; line-height: 1; font-size: 24px; font-weight: bold;">
      ┌───┬───┬───┬───┬───┬───┬───┬───┬───┬───┐
      │ A │ W │ S │ E │ D │ F │ T │ G │ Y │ H │
      │ C │C#/│ D │D#/│ E │ F │F#/│ G │G#/│ A │
      │   │Db │   │Eb │   │   │Gb │   │Ab │   │
      └───┴───┴───┴───┴───┴───┴───┴───┴───┴───┘
    </div>
    """
)

# Create buttons instead of relying on keyboard events
buttons = {}
button_layout = widgets.Layout(width='40px', height='40px')
button_box = widgets.HBox()

for key, note in keys_to_notes.items():
    button = widgets.Button(
        description=key.upper(),
        layout=button_layout,
        button_style='', # 'success', 'info', 'warning', 'danger' or ''
    )
    
    def create_handler(note_name):
        def handle_click(b):
            with output:
                clear_output(wait=True)
                print(f"Playing: {note_name} ({note_frequencies[note_name]:.2f} Hz)")
            
            # Generate and play the sound
            if waveform_selector.value == "harmonic":
                samples = create_harmonic_series(
                    fundamental_freq=note_frequencies[note_name],
                    num_harmonics=5,
                    amplitudes=[1.0, 0.5, 0.25, 0.125, 0.0625],
                    duration=1.0,
                    sample_rate=sample_rate
                )
            else:
                samples = generate_waveform(
                    frequency=note_frequencies[note_name],
                    duration=1.0,
                    waveform=waveform_selector.value,
                    sample_rate=sample_rate
                )
            
            # Play sound with explicit wait=False to make it non-blocking
            sd.play(samples, sample_rate, blocking=False)
        
        return handle_click
    
    button.on_click(create_handler(note))
    buttons[key] = button
    button_box.children = (*button_box.children, button)

# Display the interface
display(keyboard_html)
display(waveform_selector)
display(button_box)
display(output)

# Also provide a text input method with additional debug output
text_input = widgets.Text(
    placeholder='Or type keys here: A-W-S-E-D-F-T-G-Y-H-U-J-K',
    layout=widgets.Layout(width='500px')
)

def on_text_change(change):
    if 'new' in change and change['new']:
        key = change['new'].lower()
        text_input.value = ''  # Clear the input
        
        with output:
            print(f"Key pressed: {key}")  # Debug output
        
        if key in keys_to_notes:
            note = keys_to_notes[key]
            with output:
                print(f"Playing: {note} ({note_frequencies[note]:.2f} Hz)")
            
            # Generate and play the sound
            if waveform_selector.value == "harmonic":
                samples = create_harmonic_series(
                    fundamental_freq=note_frequencies[note],
                    num_harmonics=5,
                    amplitudes=[1.0, 0.5, 0.25, 0.125, 0.0625],
                    duration=1.0,
                    sample_rate=sample_rate
                )
            else:
                samples = generate_waveform(
                    frequency=note_frequencies[note],
                    duration=1.0,
                    waveform=waveform_selector.value,
                    sample_rate=sample_rate
                )
            
            # Debug output
            print(f"Generated {len(samples)} samples, max amplitude: {np.max(np.abs(samples))}")
            
            # Play sound with explicit wait=False to make it non-blocking
            sd.play(samples, sample_rate, blocking=False)

text_input.observe(on_text_change, names='value')
display(text_input)

HTML(value='\n    <div style="font-family: monospace; white-space: pre; line-height: 1; font-size: 24px; font-…

Dropdown(description='Waveform:', index=4, options=('sine', 'square', 'triangle', 'sawtooth', 'harmonic'), val…

Output()

Text(value='', layout=Layout(width='500px'), placeholder='Or type keys here: A-W-S-E-D-F-T-G-Y-H-U-J-K')